In [1]:
from keras.api.applications.vgg19 import VGG19
from keras.api.applications.vgg19 import preprocess_input, decode_predictions
from keras.api.layers import Dense, Flatten, Dropout, Activation

In [2]:
base_model = VGG19(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

In [3]:
for layer in base_model.layers:
    layer.trainable = False

In [71]:
from keras.src.utils import image_dataset_from_directory

dataset_train = image_dataset_from_directory(
    "train_data",
    image_size=(224, 224),
    batch_size=64,
    label_mode='categorical',
)

dataset_val = image_dataset_from_directory(
    "valid_data",
    image_size=(224, 224),
    batch_size=64,
    label_mode='categorical',
)

Found 29474 files belonging to 100 classes.
Found 7369 files belonging to 100 classes.


In [72]:
from tensorflow.image import convert_image_dtype
from tensorflow import float32

def convert_to_float(image, label):
    image = convert_image_dtype(image, dtype=float32)
    return image, label

In [73]:
from tensorflow.python.data import AUTOTUNE

ds_train = (
    dataset_train
    .map(convert_to_float)
    .cache()
    .prefetch(buffer_size=AUTOTUNE)
)
ds_valid = (
    dataset_val
    .map(convert_to_float)
    .cache()
    .prefetch(buffer_size=AUTOTUNE)
)

In [74]:
from keras import layers
import keras

model = keras.Sequential([

    # First Convolutional Block
    layers.Conv2D(filters=32, kernel_size=5, activation="relu", padding='same',
                  # give the input dimensions in the first layer
                  # [height, width, color channels(RGB)]
                  input_shape=[224, 224, 3]),
    layers.MaxPool2D(),

    # Second Convolutional Block
    layers.Conv2D(filters=64, kernel_size=3, activation="relu", padding='same'),
    layers.MaxPool2D(),

    # Classifier Head
    layers.Flatten(),
    layers.Dense(units=100, activation="softmax"),
])

In [75]:
model.compile(
    optimizer=keras.optimizers.Adam(epsilon=0.01),
    loss='categorical_crossentropy',
    metrics=['categorical_accuracy'],
)

model.fit(
    ds_train,
    validation_data=ds_valid,
    epochs=2,
)

Epoch 1/2
 17/461 ━━━━━━━━━━━━━━━━━━━━ 6:07 827ms/step - categorical_accuracy: 0.0346 - loss: 719.5743

KeyboardInterrupt: 

In [44]:
from tensorflow.image import decode_jpeg
from tensorflow.io import read_file
from tensorflow.image import resize
import numpy as np


img = keras.utils.load_img('train_data/Xylodon radula/2237854773-669.JPG', target_size=(224, 224))
img_array = keras.preprocessing.image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)



In [45]:
pred = model.predict(img_array)
print(pred)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
[[5.45683503e-02 3.16954509e-04 2.55142484e-04 5.13623272e-05
  4.44402033e-03 3.08218805e-05 6.90286458e-02 1.23669044e-03
  1.62825367e-04 8.82418099e-05 2.24541989e-04 8.00430626e-02
  5.63138383e-05 3.79307981e-04 5.67844254e-05 2.20641552e-04
  2.51746713e-03 1.17892504e-03 5.68814983e-04 3.10372183e-04
  5.24441339e-03 4.24901402e-04 2.12269370e-02 9.28080454e-03
  1.06142370e-05 4.39274390e-06 5.26599237e-04 3.39156538e-02
  3.39921773e-03 1.08198030e-03 9.36122378e-04 7.40642776e-04
  1.59766595e-03 4.99151392e-05 7.19995121e-04 1.09846354e-04
  6.09488599e-03 7.47591956e-04 5.12211584e-04 1.64173227e-02
  7.48150400e-04 3.42856161e-03 4.31150477e-03 8.71352404e-06
  1.24623388e-04 6.96892093e-04 7.60427862e-03 1.47717120e-03
  1.87607110e-02 3.71409714e-01 2.84691900e-03 4.15143557e-03
  4.27186396e-03 3.85851163e-04 6.54469768e-04 2.41296493e-05
  8.59408081e-03 6.66228938e-04 1.87763926e-02 3.42620013e-04
  5.57336141e-04 3.15095261e-02 

In [46]:
predicted_class = np.argmax(pred, axis=-1)

In [47]:
print(predicted_class)

[49]


In [48]:
base_model = VGG19(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

In [49]:
for layer in base_model.layers:
    layer.trainable = False

In [76]:
from keras.api.models import Model

num_classes = 100

x = base_model.output
x = Flatten()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(num_classes, activation='softmax')(x)

model_vgg = Model(inputs=base_model.input, outputs=predictions)

In [77]:
model_vgg.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [78]:
model_vgg.fit(ds_train, epochs=2)

Epoch 1/2
  5/461 ━━━━━━━━━━━━━━━━━━━━ 45:35 6s/step - accuracy: 0.0432 - loss: 53.4813

KeyboardInterrupt: 

In [68]:
img = keras.utils.load_img('train_data/Xylodon radula/2521518978-52306.JPG', target_size=(224, 224))
img_array = keras.preprocessing.image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)

pred = model_vgg.predict(img_array)
predicted_class = np.argmax(pred, axis=-1)

pred_2 = model.predict(img_array)
predicted_class_2 = np.argmax(pred_2, axis=-1)

print(predicted_class, predicted_class_2)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 183ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
[21] [98]
